In [1]:

### !!! Just change the queries.xlsx and slicers.xlsx file name as per your files name 


!jupyter nbextension enable --py powerbiclient
from powerbiclient import Report, models
from io import StringIO
from ipywidgets import interact
import requests
import pandas as pd
from powerbiclient.authentication import MasterUserAuthentication
import json
import sql_metadata
import itertools
import psycopg2
import time
import re
import numpy
from decimal import Decimal
import csv
import sys

## Function Report Render Helper Functions:
## Function - 01
def HelperRender(report):
    def loaded_callback(event_details):
        print('Report is loaded')
    report.on('loaded', loaded_callback)
    def rendered_callback(event_details):
        print('Report is rendered')
    report.on('rendered', rendered_callback)
    def error_callback(event_details):
        print('error', event_details)
    report.on('error', error_callback)

## Function - 02
## User Input -> Report instance 
## input -> bi credential + Report and group ID
def Input_BI(username,pas,report_id,group_id):
    master_user_auth = MasterUserAuthentication(username, pas)
    token_with_master_user = master_user_auth.get_access_token()
    report = Report(group_id=group_id, report_id=report_id, access_token = token_with_master_user,auth=master_user_auth)
    return report

def Input_DataBase(db_name,host,user,password,port):
    conn = psycopg2.connect(dbname= db_name, host=host, 
    port= port, user= user, password=password)
    
    return conn

## Function -03:
## Slicer Combination Creation
## dic -> query where condition
## Function -> Combinations of slicers



def SlicerCombo(csv_file,dic):
    
    df = pd.read_excel(csv_file,sheet_name='Sheet1')

    df.fillna('a',inplace=True)
    ## Slicer dictionary with their values given by user
    slicerdict = {}
    slicerdict = df.to_dict('list')
    for key,val in slicerdict.items():
        val = [i for i in val if i!='a']
        val = [int(i) if isinstance(i,float) else i for i in val  ]
        slicerdict[key] = val
    Final_dict = {}
    for key in dic.keys():
        Final_dict[key] = slicerdict[key]
    MergeSlicerList = []
    for key,value in Final_dict.items():
        MergeSlicerList.append(value)

    ## all combinationList
    CombinationSlicer = []
    tc = []
    CombinationSlicer = list(itertools.product(*MergeSlicerList))
    combinationSlicer =  [list(ele) for ele in CombinationSlicer]
    

        

    return combinationSlicer

def DataFrame2dict2(df):
    ddict = {}
    x,y = df.shape
    row = 0
    while row < x:
        data = list(df.iloc[row])
        ls = [re.sub('[^.,a-zA-Z0-9 \n\.]','', string)  if type(string) == str else string for string in data[3:] ] ## change regex for .
        ls = [ele.strip() if type(ele) == str else ele for ele in ls] ## extra space remove
        ls = [float(ele) if type(ele) == str and ele.replace('.','').isdigit() else ele for ele in ls] ## string with digits float
        ls = [float(ele) if (type(ele) == numpy.int64 or type(ele) == int or type(ele) == numpy.float64) else ele for ele in ls]
        ls = [float(ele) if (type(ele)!=str and type(ele)!=float and type(ele)!= type(None)) else ele for ele in ls ]
        key = str(data[0])+str(data[1])+str(data[2])
        ddict[key] = ls
        row = row+1
    return ddict

def KPIValidate(dataBI,datasql,combination,dic):
    error_list = []
    col = datasql.columns[0]
    
    val_sql = float(datasql[col].values)
    dataBI.columns = [ele.lower() for ele in list(dataBI.columns)]
    try:
        val_bi = float(dataBI[col].values)
    except:
        print('Kindly Give KPI column name as per the DAX query')
        #print('\n')
        error_list.append('Kindly Give KPI column name as per the DAX query')
        return error_list
        
    if abs(val_bi-val_sql) < 2:
        print('Values Matching')
        #print('\n')
        return error_list
    else:
        print('Not Matched')
        print('Sql Value for this KPI->',val_sql)
        print('Power BI Value for this KPI->',val_bi)
        #print('\n')
        st = "Value->"+str(dataBI[col]) + "Column->" + str(col) + "Combinations->"+ str(combination)
        error_list.append(st)
        return error_list
    
    


def DataFrame2dict(df):
    ddict = {}
    x,y = df.shape
    row = 0
    while row < x:
        data = list(df.iloc[row])
        ls = [re.sub('[^.,a-zA-Z0-9 \n\.]','', string)  if type(string) == str else string for string in data[1:] ] ## change regex for
        ls = [ele.strip() if type(ele) == str else ele for ele in ls] ## extra space remove
        ls = [float(ele) if type(ele) == str and ele.replace('.','').isdigit() else ele for ele in ls] ## string with digits float
        ls = [float(ele) if (type(ele) == numpy.int64 or type(ele) == int or type(ele) == numpy.float64) else ele for ele in ls]
        ls = [float(ele) if (type(ele)!=str and type(ele)!=float and type(ele)!= type(None)) else ele for ele in ls ]
        ddict[data[0]] = ls
        row = row+1
    return ddict
    
## Function -04
## Filter Creation + Filter Apply
## Filter Apply Function
import time
def bi_filter(table_name , column_name , value):
    ret_val = {
      '$schema': "http://powerbi.com/product/schema#basic",
      'target': {
          'table': table_name,
          'column': column_name
      },
      'operator': "In",
      'values': [value]
    }
    
    return ret_val


## visual_name = visual_ID
def FilterValues(slicer_combo,dic,idx,conn,report,active_page_name,sql,visual,error_list):
    
    report.remove_filters()
    print('error_list->',error_list)
    
    if idx == len(slicer_combo):
        print('Validation Done')
        return 
    
    print('Applying Filters->',slicer_combo[idx])
          
    for i,ele in enumerate(dic):
        (table,col) = dic[ele],ele
        value = slicer_combo[idx][i] ## First filter

        if type(value) == str and value.isdigit():
            value = int(value)

        fltr = bi_filter(table , col , value)

        report.update_filters([fltr])
        #time.sleep(3)
        
    
    print(' ALL filters applied')
    ## Data Dumps Here
    time.sleep(10)
    summarized_exported_data = report.export_visual_data(active_page_name, visual['name'], rows = 1000)
    
    ## Getting Sql data
    sql_updated = ChangeQuery(sql,slicer_combo[idx])
    
    try:
        df_sql = GetSqlData(conn,sql_updated)
        df_sql = df_sql.fillna(value=0)
        print('SQL Query Executed for {} test case'.format(idx+1))
        
    except Exception as e:
        #error_list.append("sql error exist::" + str(e) + " Combination->"+ str(slicer_combo[idx]))
        print('Error Occured in the given Query for this combination->',slicer_combo[idx])
        print('Going to next combination')
        return
    
    time.sleep(5)
    ## reading data in csv format
    
    data = StringIO(summarized_exported_data)

    df_BI = pd.read_csv(data, sep=",")
    df_BI = df_BI.fillna(value=0)

    
    if visual['type'] == 'kpi':
        try:
            ans = KPIValidate(dataBI = df_BI,datasql = df_sql,combination = slicer_combo[idx])
            error_list.append(ans)
        except:
            pass
    
    else:
        
        ## dictionary for BI and sql
        dbi = DataFrame2dict(df_BI)
        dsql = DataFrame2dict(df_sql)
        
        ## validation for length == 0
        if (df_sql.shape[0]) == 0 and df_BI.shape[0] == 0:
            print('Both the sources have length equal to zero Taking this as Values Matching')
            error_list.append([])
            print('\n')
        elif (df_sql.shape[0] == 0 and df_BI.shape[0] != 0):
            print("Sql doesnt contain any data but power BI contains for combination->",slicer_combo[idx])
            st = "Sql doesnt contain any data but power BI contains for combination->"+str(slicer_combo[idx])
            error_list.append([st])
            print('\n')
        elif (df_sql.shape[0] != 0 and df_BI.shape[0] == 0):
            print("Power BI doesnt contain any data but Sql contains for combination->",slicer_combo[idx])
            st = "Power BI doesnt contain any data but Sql contains for combination->"+str(slicer_combo[idx])
            error_list.append([st])
            print('\n')
        else:
            if df_sql.shape[0] > len(dsql):
                dsql2 = DataFrame2dict2(df_BI)
                dbi2 = DataFrame2dict2(df_sql)
                print("Repeated Values Occured going to case 2")
                try:
                    ans = ValidationCheck(dsql2,dbi2,slicer_combo[idx])
                    error_list.append(ans)
                except:
                    pass
            else:
                try:
                    ans = ValidationCheck(dbi,dsql,slicer_combo[idx])
                    error_list.append(ans)
                except:
                    pass
            
    FilterValues(slicer_combo,dic,idx+1,conn,report,active_page_name,sql,visual,error_list)

    
def ValidateVisual(dic,slicer_combo,report,active_page_name,query,error_list,conn,visual,idx,imerror):
    
    print('Applying Filters->',slicer_combo)      
    for i,ele in enumerate(dic):
        (table,col) = dic[ele],ele
        value = slicer_combo[i] ## First filter Value

        if type(value) == str and value.isdigit():
            value = int(value)

        fltr = bi_filter(table , col , value)

        report.update_filters([fltr])
        #time.sleep(3)
        
    time.sleep(10)
    print(' ALL filters applied')
    summarized_exported_data = report.export_visual_data(active_page_name, visual['name'], rows = 1000)
    ## Getting Sql data
    sql_updated = ChangeQuery(query,slicer_combo)
    
    df_sql = GetSqlData(conn,sql_updated)
    print('SQL Query Executed for {} test case'.format(idx+1))

    
    time.sleep(5) ## Fetching time for SQL
    
    data = StringIO(summarized_exported_data)

    df_BI = pd.read_csv(data, sep=",")
    df_BI = df_BI.fillna(value=0)
    df_sql = df_sql.fillna(value=0)
    
    if visual['type'] == 'kpi':
        try:
            ans = KPIValidate(dataBI = df_BI,datasql = df_sql,combination = imerror)
            error_list.append(ans)
            return 
        except:
            print('Some Error Occured in this widget for this combination->',imerror)
    
    else:
        
        ## dictionary for BI and sql
        dbi = DataFrame2dict(df_BI)
        dsql = DataFrame2dict(df_sql)
        
        ## validation for length == 0
        if (df_sql.shape[0]) == 0 and df_BI.shape[0] == 0:
            print('Both the sources have length equal to zero Taking this as Values Matching')
            error_list.append([])
            #print('\n')
            return 
        
        elif (df_sql.shape[0] == 0 and df_BI.shape[0] != 0):
            print("Sql doesnt contain any data but power BI contains for combination->",imerror)
            st = "Sql doesnt contain any data but power BI contains for combination->"+str(imerror)
            error_list.append([st])
            #print('\n')
            return 
        
        elif (df_sql.shape[0] != 0 and df_BI.shape[0] == 0):
            print("Power BI doesnt contain any data but Sql contains for combination->",imerror)
            st = "Power BI doesnt contain any data but Sql contains for combination->"+str(imerror)
            error_list.append([st])
            #print('\n')
            return 
        
        else:
            if df_sql.shape[0] > len(dsql):
                dsql2 = DataFrame2dict2(df_BI)
                dbi2 = DataFrame2dict2(df_sql)
                print("Repeated Values Occured going to case 2")
                try:
                    ans = ValidationCheck(dsql2,dbi2,imerror)
                    error_list.append(ans)
                    return 
                
                except:
                    print('Some error occured in this widget for this Visual->',imerror)
            else:
                try:
                    ans = ValidationCheck(dbi,dsql,imerror)
                    error_list.append(ans)
                    return
                
                except:
                    pass
    
    
## Function -04:
## Where Condition Dic creation :
def GetWhereSql(queries):
    dic = {} # Dictionary to store column name as key and table name as value
    try:
        sub_query = sql_metadata.Parser(queries).subqueries
        cols = sql_metadata.Parser(sub_query[')']).columns_dict['where']
    except:
        cols = sql_metadata.Parser(queries).columns_dict['where']


    for ele in cols:
        if '.' in ele:
            ans = ele.split('.')[1:]
            dic[ans[1]] = ans[0]
    return dic

## Function -05
## applied sql with new where conditions
def single_quotes(a):
    b = "'"+ a+"'"
    return b

def ChangeQuery(sql,combination):
    applycombo = []
    for ele in combination:
        if type(ele) == str:
            applycombo.append(single_quotes(ele))
        else:
            applycombo.append(ele)
            
    updated_sql = sql.format(*applycombo)
    return updated_sql

## Function -06
## getting visual data and information
def get_active_page(self):
    
    # Get list of pages
    pages = self.get_pages()
    active_page = {}
    for page in pages:
        
        # Get active page
        if page['isActive'] == True:
            active_page = page
            break
            
    return active_page

## Function - 07
def GetSqlData(conn,query):
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
    cursor.execute(query)
    headers = [i[0] for i in cursor.description]
    df_sql = pd.DataFrame(data, columns = headers)

    return df_sql

## Get Visual_ID From Visual_name
def Extract_Visual(report,visual_name):
    active_page = get_active_page(report)
    active_page_name = active_page['name']
    
    # Get list of visuals on active page
    visuals = report.visuals_on_page(active_page_name)
    visual_name = visual_name.lower()
    visual_name = visual_name.replace(' ','')
    for vis in visuals:
        
        try:
            b = vis['title']
            b =  b.lower()
            b = b.replace(' ','')
            if b == visual_name:
                a = (vis['name'])
        except:
            pass

    return a

## dictionary for validation
def DataFrame2dict(df):
    ddict = {}
    x,y = df.shape
    row = 0
    while row < x:
        data = list(df.iloc[row])
        ls = [re.sub('[^.,a-zA-Z0-9 \n\.]','', string)  if type(string) == str else string for string in data[1:] ]
        ddict[data[0]] = ls
        row = row+1
    return ddict

def ValidationCheck(dbi,dsql,combination):
    
    error_list = []

    ans = [dbi if len(dbi)<=len(dsql) else dsql][0]
    flag = 0
    for key in ans.keys():
        if key in dbi.keys() and key in dsql.keys():
            val_bi = dbi[key]
            val_sql = dsql[key]
            #flag = 0
            for item in zip(val_bi,val_sql):
                if type(item[0])== float or type(item[1]) ==float:

                    if abs(item[0]-item[1]) < 1:
                        #print('Matching')
                        pass
                    else:
                        #print('Values Not Matched Check this Row->',key)
                        #print('Values Not Matched')
                        flag = 1
                elif flag ==0 and (type(item[0])==str or type(item[1])==str):
                    if item[0] == item[1]:
                        #print('Matching')
                        pass
                    else:
                        #print('Values Not Matched Check this Row->',key)
                        #print('Not Matching')
                        flag = 1
    if flag == 0:
        print('all widgets values passed for combination->',combination)
        #print('\n')
        return error_list
    else:
        print('some values are not matching in this widget ,Failed for this combination->',combination)
        #print('\n')
        error_list.append('check this widget for this combination->'+str(combination))
        return error_list
            

## Loading config file 
with open('config.json') as config_file:
    data = json.load(config_file)


report = Input_BI(username = data['username'], pas =  data['password_bi'] ,group_id= data['group_id'],report_id= data['report_id'])
#HelperRender(report)
report


Enabling notebook extension powerbiclient/extension...
      - Validating: ok


Authenticating master user.
Master user authentication successfully completed.


Report()

In [ ]:
sys.stdout = open("Validation_Summary.txt", "w")
conn = Input_DataBase(db_name = data['dbname'],host = data['host'],user = data['user'],password = data['password_db']
                      ,port = data['port'])
df_error = pd.DataFrame()
pages = report.get_pages()
error_dict = {}
stack = []
validation_summary = []

sheet_names = [ele['displayName'] for ele in pages]
itr = 0
for index,i in enumerate(sheet_names):
    df_query = pd.read_excel('queries_under_development-5.xlsx',sheet_name = i)
    visual_name = df_query.columns.tolist()
    input("Please Select {} page to validate all its visuals and Press Enter if you have selected the given page...".format(pages[index]['displayName']))
    print('\n')
    print('Validating visuals for the {} page'.format(pages[index]['displayName']))
    print('\n')
    for visual in visual_name:
        if 'Unnamed' not in visual :
            visual_title = visual
            print('Validating for the Visual->',visual)
            query = df_query[visual].values[0]
            dic = GetWhereSql(query)
            combo = []
            combo = SlicerCombo('slicers-3.xlsx',dic)
            active_page_name = pages[index]['name']
            visual_ID = Extract_Visual(report,visual_title)
            visuals = report.visuals_on_page(active_page_name)
            for e,vis in enumerate(visuals):
                if vis['name'] == visual_ID:
                    visual_dict = visuals[e]

            error_list = []
            for idx,slicer_combo in enumerate(combo[:2]):
                imerror = []
                for j,e in enumerate(dic.keys()):
                    imerror.append(str(e)+"->"+str(combo[0][j]))
                    
                report.remove_filters()
                try:
                    conn = Input_DataBase(db_name = data['dbname'],host = data['host'],user = data['user'],password = data['password_db'],port = data['port'])
                    ValidateVisual(dic,slicer_combo,report,active_page_name,query,error_list,conn = conn,visual = visual_dict,idx = idx,imerror=imerror)
                except:
                    error_list.append(['Error Occured in the query Execution for this slicers->' + str(imerror)])
                    print('Please check the query of this {} visual'.format(visual))
                report.remove_filters()
            tc = len(error_list)
            tp = 0
            for i in error_list:
                if type(i) == list:
                    if len(i) == 0:
                        tp += 1
            print('Total Case Run is {} and total case passed {}'.format(tc,tp))
            st = 'Total Case Run is {} and total case passed {}'.format(tc,tp)
            validation_summary.append(st)
            print('Writing Errors to the Error_list')
 
            #field = [visual_dict['title']]
            error_list = [ele for ele in error_list if len(ele)!=0]
            print('Error For this {} visual-> is {}'.format(visual,error_list))
            error_dict[visual] = error_list
            stack.append(error_dict)
            error_dict = {}
            print('\n')

print('\n')
print('Validation Done')
df = pd.DataFrame(stack)
for i,col in enumerate(df.columns.tolist()):
    if i!=0:
        df[col][0] = df[col][i]
        df[col][i] = None
df = df.dropna()
df.to_csv('Error_list_file.csv')
print('Check Error_list_file for Error Information')

Please Select Risk Performance page to validate all its visuals and Press Enter if you have selected the given page...


In [ ]:
conn = Input_DataBase(db_name = data['dbname'],host = data['host'],user = data['user'],password = data['password_db']
                      ,port = data['port'])
df_error = pd.DataFrame()
pages = report.get_pages()
error_dict = {}
stack = []
validation_summary = []

sheet_names = [ele['displayName'] for ele in pages]

In [ ]:
df_query 

In [ ]:
query = df_query['Lives Across Disease Cohort'].values[0]
dic = GetWhereSql(query)
combo = []
combo = SlicerCombo('slicers-3.xlsx',dic)

In [ ]:
dic

In [ ]:
combo[0]

In [ ]:
imerror = []
for idx,e in enumerate(dic.keys()):
    imerror.append(str(e)+"->"+str(combo[0][idx]))

In [ ]:
imerror